# Peripheral showcase: RPI sense hat in CHI@Edge

Welcome to this Jupyter notebook guide on using the [Raspberry Pi Sense Hat](https://www.raspberrypi.com/documentation/accessories/sense-hat.html) within CHI@Edge, a sensor expansion board specially designed for Raspberry Pi that houses an onboard gyroscope, accelerometer, magnetometer, barometer, temperature and humidity sensor, etc.. This artifact will walk you through the steps to access and utilize the sense hat for your edge computing experiments.. 

Please check out our extensive [documentation](https://chameleoncloud.gitbook.io/chi-edge/device-enrollment/peripherals-and-device-profiles) on how to add peripherals to CHI@Edge devices

In the following example we'll show how to use the sense hat to capture sensor data.

In [ ]:
import chi
chi.use_site("CHI@Edge")

In [ ]:
chi.set("project_name", "PLACEHOLDER")

In [ ]:
from chi import container
from chi import lease

# backwards compa
from chi import context
context.version = "0.17"

## Creating the Lease

To access the sense hat, we need to make a lease for the specific device that the camera is currently attached to. The device `acaces-byod-demo-01` is specifically set up with the proper kernel and driver options to enable support for the waveshare sense hat

In [ ]:
import os

# get your username, just used to name leases something identifiable
username = os.environ.get("USER")

# machine name refers to the "type" of device
machine_name = "raspberrypi4-64"

# Reserving the specific device to which the Wavesharee sense Pi-hat is attached
device_name = "acaces-byod-demo-01"

# get dates for lease start and end
start, end = lease.lease_duration(days=1)

# make a unique name for the lease
lease_name = f"{username}-{machine_name}-{start}"

reservations = []
lease.add_device_reservation(reservations, count=1, machine_name=machine_name, device_name=device_name)
container_lease = lease.create_lease(lease_name, reservations, start, end)
lease_id = container_lease["id"]

print(f"created lease with name {lease_name} and uuid {lease_id}, waiting for it to start. This can take up to 60s.")
lease.wait_for_active(lease_id)
print("Done!")

In [ ]:
print("Requesting container ... This may take a while as the large container image is being downloaded")

# Set a name for the container. Because CHI@Edge uses Kubernetes, ensure that underscores aren't in the name
container_name = f"tutorial-acaces-sensehat".replace("_","-")

try:
    my_container = container.create_container(
        container_name,
        image="docker.io/shermanm/sense-hat-test:alpine1-latest",
        device_profiles=["pi_sensehat"],
        reservation_id=lease.get_device_reservation(lease_id),
    )
except RuntimeError as ex:
    print(ex)
    print(f"please stop and/or delete {container_name} and try again")
else:
    print(f"Successfully created container: {container_name}!")

## Destroying the container after use

In [ ]:
chi.container.destroy_container(my_container.uuid)